In [2]:
from neo4j import GraphDatabase

In [ ]:
from regraph.graphs import Graph

In [ ]:
import networkx

In [ ]:
from regraph import NXGraph, TypedNeo4jGraph, Rule

In [7]:
from regraph.attribute_sets import IntegerSet, RegexSet

In [4]:
%matplotlib inline

## connecting to Neo4j sandbox

In [1]:
uri="bolt://3.82.98.115:7687"
pwd="explosives-circumstance-sample"

In [4]:
driver = GraphDatabase.driver(uri, auth=("neo4j",pwd))

In [5]:
query= "Match (n) RETURN COUNT(n)"

In [6]:
with driver.session() as session:
    result = session.run(query)
    print(result.single()["COUNT(n)"])
    

0


In [7]:
from py2neo import Graph, Node, Relationship

In [8]:
graph = Graph(uri, auth=("neo4j",pwd))

In [9]:
graph

Graph('bolt://3.82.98.115:7687')

## Load all csv files to neo4j

In [ ]:
//count data rows in features.csv (headers)
LOAD CSV WITH HEADERS FROM 'features.csv' AS row
RETURN count(row);

## Create and modify a schema-aware graph object

In [11]:
# define schema graph elements
schema_nodes= [
    ("project", {"course": RegexSet.universal()}),
    ("courseName", {"course": RegexSet.universal()}),
    ("student", {"roles": RegexSet.universal()}),
    ("user", {"roles": RegexSet.universal()}),
    ("ethicalHacker", {"roles": RegexSet.universal()}),
    ("attacker", {"roles": RegexSet.universal()}),
    ("app", {"application": RegexSet.universal()}),
    ("system", {"application": RegexSet.universal()}),
    ("tools", {"application": RegexSet.universal()}),
    ("technique", {"concept": RegexSet.universal()}),
    ("vulnerability", {"concept": RegexSet.universal()}),
    ("attack", {"concept": RegexSet.universal()}),
    ("function", {"concept": RegexSet.universal()}),
    ("feature", {"concept": RegexSet.universal()}),
]

In [10]:
schema_edges =[
    ("project", "system", {"type":{"has_prereq"}}),
    ("project", "concept", {"type":{"has_prereq"}}),
    ("system", "attack", {"type":{"can_expose"}}),
    ("function", "attack", {"type":{"can_expose"}}),
    ("app", "feature", {"type":{"has_a"}}),
    ("tools", "function", {"type":{"has_a", "can_analyze"}}),
    ("tools", "apps", {"type":{"can_analyze"}}),
    ("tools", "vulnerability", {"type":{"can_analyze"}}),
    ("tools", "tools", {"type":{"implements"}}),
    ("system", "vulnerability", {"type":{"can_expose"}}),
    ("function", "vulnerability", {"type":{"can_expose"}}),
    ("feature", "attack", {"type":{"can_cause"}}),
    ("attacker", "vulnerability", {"type":{"can_exploit"}}),
    ("attacker", "feature", {"type":{"can_exploit"}}),
    ("ethicalHacker", "vulnerability", {"type":{"can_analyze"}}),
    ("ethicalHacker", "tools", {"type":{"uses"}}),
    ("ethicalHacker", "app", {"type":{"can_exploit"}}),
    ("ethicalHacker", "technique", {"type":{"implements"}}),
    ("user", "system", {"type":{"uses"}}),
    ("user", "vulnerability", {"type":{"can_expose"}}),
    ("student", "system", {"type":{"uses"}}),
    ("student", "feature", {"type":{"can_analyze"}}),
    ("student", "function", {"type":{"implements"}}),
    ("student", "technique", {"type":{"implements"}}), 
    ("student", "tools", {"type":{"uses"}}),
    
]

In [17]:
# Define data graph elements
data_nodes = [
    ("Project3"),
    ("HomeNetwork"),
    ("LandAttack"),
    ("TeamDefense"),
    ("TCPSYNPacket"),
    ("AttackHost"),
    ("Snort"),
    ("SnortRules")
]
data_edges = [
    ("Project3", "HomeNetwork", {"type": "has_prereq"}),
    ("TeamDefense", "HomeNetwork", {"type": "uses"}),
    ("TeamDefense", "SnortRules", {"type": "implements"}),
    ("SnortRules", "LandAttack", {"type": "can_expose"}),
    ("HomeNetwork", "LandAttack", {"type": "can_expose"}),
    ("Snort", "SnortRules", {"type": "has_a"}),
    ("TCPSYNPacket", "LandAttack", {"type": "can_cause"}),
    ("AttackHost", "TCPSYNPacket", {"type": "can_exploit"}),
]

# Provide typing of the data by the schema
data_typing = {
    "Project3": "project",
    "TeamDefense": "student",
    "HomeNetwork": "system",
    "LandAttack": "attack",
    "TCPSYNPacket": "feature",
    "AttackHost": "attacker",
    "Snort": "tools",
    "SnortRules": "function",
}

In [16]:
# Create a schema-aware graph
graph = TypedNeo4jGraph(
    uri="bolt://3.82.98.115:7687",
    user="neo4j",
    password="explosives-circumstance-sample",
    data_graph={"nodes": data_nodes, "edges": data_edges},
    schema_graph={"nodes": schema_nodes, "edges": schema_edges},
    typing=data_typing,
    clear=True)

In [ ]:
# add new data to schema-aware graph 
graph.add_data_node("PacketDecoder", typing="tools", attrs={"category": "application"})
graph.add_data_edge("Snort", "PacketDecoder", {"type": "implements"})
graph.add_data_node("DetectionEngine", typing="tools", attrs={"category": "application"})
graph.add_data_edge("Snort", "DetectionEngine", {"type": "implements"})